In [2]:
category, date, page = 100, 20191121, 1

url = "https://news.naver.com/main/list.nhn?\
mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)

In [3]:
driver = webdriver.Chrome()
driver.get(url)

In [16]:
articles = driver.find_elements_by_css_selector(".list_body > ul > li")
len(articles)

20

In [19]:
title = articles[0].find_element_by_css_selector("dt:not(.photo) > a").text
link = articles[0].find_element_by_css_selector("dt:not(.photo) > a")\
.get_attribute("href")
title, link

('[이슈 따라잡기] 황교안, 이틀째 단식투쟁…“절체절명 국가위기 막겠다”',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=374&aid=0000198702')

In [22]:
response = requests.get(link)
dom = BeautifulSoup(response.content, "html.parser")

In [27]:
content = dom.select_one("#articleBodyContents").text\
.replace("\n", "").replace("\'", "").replace("- ", "").split("{}")[1].strip()
content

'■ 용감한 토크쇼 직설진행 : 임종윤출연 : 김수민 시사평론가Q. 황교안, 삭발에 단식까지…그 다음 순서는?황교안 단식 시작…"죽기를 각오한 무기한 투쟁"황교안 대표, 지소미아 파기·패스트트랙 강행 항의황교안, 영수회담 제안 거부되자 단식 결심한 듯황교안, 총체적 리더십 위기 속 단식투쟁 초강수황교안 단식에…"하지마라" vs "목숨 건 것"홍준표 "文대통령, 황교안 단식에 미동도 안할 것"민주당 "황교안 단식, 정치초보의 떼쓰기에 불과"바른미래 "黃, 정치수준 얼마나 더 떨어뜨릴 거냐"文대통령, 강기정에 "황교안, 집앞에 온 손님…찾아봬라"靑강기정, 황교안 만나 단식 만류…"옳은 방향 아냐Q. 황교안 단식, 제2의 패스트트랙 사태 초래할까?黃 단식에 정국 경색 조짐…패스트트랙 충돌 우려한국당 "불법 패스트트랙 중단하고 원점 논의해야"黃, 무기한 단식 돌입…한국당 동조 단식도 검토선거법 11월27일…공수처법 12월3일 부의 예정민주당 "황교안, 단식 아닌 협상해야"與 패트 일방처리 시사하며 한국당 압박황교안 단식에 패트 여야4당 공조 복원될까?Q. "말만 좀 더 잘하면"…靑고민정, 탁현민에 뿔?고민정 "아수라장 걱정이었지만 작은 대한민국"고민정 "대통령에게 죄송하지만 정말 괜찮았다"김어준 "도떼기 시장, 기획 잘못"탁현민 "국민과의 대화, 나였다면 안했다"(자세한 내용은 동영상을 시청하시기 바랍니다.)   ☞ 의견 있는 경제채널 SBSCNBC [홈페이지]☞ 경제를 실험한다~ #머니랩 [네이버 TV] 저작권자 SBS미디어넷 & SBS I&M 무단전재-재배포 금지'

In [39]:
def get_articles(category, date, page):
    url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    articles = driver.find_elements_by_css_selector(".list_body > ul > li")
    datas = []
    for article in articles:
        title = article.find_element_by_css_selector("dt:not(.photo) > a").text
        link = article.find_element_by_css_selector("dt:not(.photo) > a").get_attribute("href")
        response = requests.get(link)
        dom = BeautifulSoup(response.content, "html.parser")
        content = dom.select_one("#articleBodyContents").text.replace("\n", "").replace("\'", "").replace("- ", "").split("{}")[1].strip()
        datas.append({
            "title": title,
            "link": link,
            "content": content,
            "category": category,
        })
    driver.quit()
    return pd.DataFrame(datas)

In [40]:
categories = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

In [41]:
dfs = []
date = 20191121
for category in categories:
    print(categories[category], end=" ")
    for page in range(1, 6):
        print(page, end=" ")
        df = get_articles(category, date, page)
        dfs.append(df)
    print()
articles_df = pd.concat(dfs, ignore_index=True)
articles_df.tail()

정치 1 2 3 4 5 
경제 1 2 3 4 5 
사회 1 2 3 4 5 
생활/문화 1 2 3 4 5 
세계 1 2 3 4 5 
IT/과학 1 2 3 4 5 


,category,content,link,title
595,105,"[디지털데일리 채수웅기자] 과학기술정보통신부(장관 최기영)는 21일 KT, SK텔레...",https://news.naver.com/main/read.nhn?mode=LSD&...,"과기정통부-통신사, 재난방지 대책 추진현황 점검"
596,105,"게임 기획 단계부터 타격감 위주로 설계(지디넷코리아=김한준 기자)""슬랩잇은 기획 ...",https://news.naver.com/main/read.nhn?mode=LSD&...,STAC 2019 게임 수상작 '슬랩잇' 고등학생 퀄리티 넘어섰다
597,105,"과기정통부, 22일자로 IBS 신임 원장 임명[아이뉴스24 최상국 기자] 기초과학연...",https://news.naver.com/main/read.nhn?mode=LSD&...,"노도영 IBS 신임 원장 임명, ""IBS 개혁 · 중이온가속기 성공"" 임무"
598,105,병역 대체복무제도 개선 확정…중소·중견기업에 배정 늘려(세종=연합뉴스) 신선미 기자...,https://news.naver.com/main/read.nhn?mode=LSD&...,2025년 전문연구요원 석사 300명 감축…박사는 1천명 유지(종합)
599,105,엔지니어들이 SOC실증연구센터에서 스마트 건설 방식으로 공사를 진행하고 있다. 제...,https://news.naver.com/main/read.nhn?mode=LSD&...,"SKT, ‘5G+건설·기계’ 드론 띄워 측량, 굴삭기로 정밀공사 ‘착착’"


In [42]:
articles_df.to_csv("article.csv", index=False, encoding="utf-8-sig")